In [132]:
import warnings
import pandas as pd
import re
import json
import spacy
warnings.filterwarnings('ignore')

In [145]:
df = pd.read_csv("../data/MultiLabelSongs_EN.csv")
df

,artist,genre,title,album,lyrics,labels
0,Nirvana,Rock,You Know You’re Right,Nirvana,I will never bother you\nI will never promise ...,"Calmness, Sadness"
1,Damian Marley,Reggae,Here We Go,Stony Hill,Here we go\nMy big ego is gonna get me in trou...,"Power, Tension"
2,The Mission UK,Rock,Jade,Another Fall from Grace,Adorned with Stars\nWith her deft allusion and...,"Amazement, Calmness, Solemnity, Tenderness"
3,UB40,Reggae,Food For Thought,Signing Off,"Ivory Madonna, dying in the dust\nWaiting for ...","Joyful activation, Sadness, Tension"
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,I was totin' my pack along the dusty Winnemucc...,"Amazement, Calmness, Joyful activation"
...,...,...,...,...,...,...
1105,Run-D.M.C.,Hip-Hop,King of Rock,King of Rock,"I'm the king of rock, there is none higher\nSu...","Joyful activation, Power"
1106,The Beatles,Rock,Yesterday,Help!,Yesterday\nAll my troubles seemed so far away\...,"Nostalgia, Sadness, Tenderness"
1107,The Sundays,Rock,Love,Blind,Picture myself as a thin white child\nBack to ...,"Calmness, Power, Tenderness"
1108,The Cure,Rock,Killing an Arab,Boys Don’t Cry,Standing on the beach\nWith a gun in my hand\n...,"Power, Solemnity, Tension"


In [146]:
print(pd.Series({
    "Number of observations with missing values": df.isnull().any(axis=1).sum(),
    "Number of duplicate observations": df.duplicated().sum()
}).to_string())

Number of observations with missing values    0
Number of duplicate observations              0


In [147]:
df['labels'] = df['labels'].str.split(',\s*')
df

,artist,genre,title,album,lyrics,labels
0,Nirvana,Rock,You Know You’re Right,Nirvana,I will never bother you\nI will never promise ...,"[Calmness, Sadness]"
1,Damian Marley,Reggae,Here We Go,Stony Hill,Here we go\nMy big ego is gonna get me in trou...,"[Power, Tension]"
2,The Mission UK,Rock,Jade,Another Fall from Grace,Adorned with Stars\nWith her deft allusion and...,"[Amazement, Calmness, Solemnity, Tenderness]"
3,UB40,Reggae,Food For Thought,Signing Off,"Ivory Madonna, dying in the dust\nWaiting for ...","[Joyful activation, Sadness, Tension]"
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,I was totin' my pack along the dusty Winnemucc...,"[Amazement, Calmness, Joyful activation]"
...,...,...,...,...,...,...
1105,Run-D.M.C.,Hip-Hop,King of Rock,King of Rock,"I'm the king of rock, there is none higher\nSu...","[Joyful activation, Power]"
1106,The Beatles,Rock,Yesterday,Help!,Yesterday\nAll my troubles seemed so far away\...,"[Nostalgia, Sadness, Tenderness]"
1107,The Sundays,Rock,Love,Blind,Picture myself as a thin white child\nBack to ...,"[Calmness, Power, Tenderness]"
1108,The Cure,Rock,Killing an Arab,Boys Don’t Cry,Standing on the beach\nWith a gun in my hand\n...,"[Power, Solemnity, Tension]"


In [148]:
print(df['lyrics'][9])

"If I only could, I'd be running up that hill
If I only could, I'd be running up that hill"

It doesn't hurt me
You want to feel how it feels?
You want to know, know that it doesn't hurt me?
You want to hear about the deal I'm making?
You (be running up that hill)
You and me (be running up that hill)

And if I only could
Make a deal with God
I'd get him to swap our places
Be running up that road
Be running up that hill
Be running up that building
If I only could, oh

You don't want to hurt me
But see how deep the bullet lies
Unaware that I'm tearing you asunder
There's a thunder in our hearts, baby
So much hate for the ones we love
Tell me, we both matter, don't we?
You (be running up that hill)
You and me (be running up that hill)
You and me won't be unhappy

And if I only could
Make a deal with God
I'd get him to swap our places
Be running up that road
Be running up that hill
Be running up that building
If I only could, oh

C'mon, baby, c'mon, c'mon darling
Let me steal this moment f

In [149]:
with open('../data/useless_words_lowercase.txt', 'r') as file:
    content = file.read()
useless_words = [word.strip() for word in content.replace('\n', ',').split(',') if word.strip()]
print(len(useless_words))

779


In [150]:
# Removing all words from the useless_words list, regardless of case (using word delimiters (\b) to avoid removing fragments of words)
df['lyrics'] = df['lyrics'].str.replace(r'(?i)\b(' + '|'.join(map(re.escape, useless_words)) + r')\b', '', regex = True)
df['lyrics'] = df['lyrics'].str.replace(r'\s+', ' ', regex=True).str.strip() # multiple spaces => single space

# Replacing any combination of spaces, commas, or parentheses with a single space
df['lyrics'] = df['lyrics'].str.replace(r'(?:[\s,()]+)', ' ', regex=True)
df['lyrics'] = df['lyrics'].str.replace(r'\s+', ' ', regex=True).str.strip() # multiple spaces => single space

# Replacing words ending in "n'" with the full version with "ng"
df['lyrics'] = df['lyrics'].str.replace(r"\b(\w*?)n'(?![a-zA-Z])", r"\1ng", regex=True)

# Removing musical structure tags like [Verse 1], {Chorus}, ...
df['lyrics'] = df['lyrics'].str.replace(
    r'(?i)\{?\b(?:intro|verse\s*\d*|chorus|bridge|outro|pre-chorus|post-chorus|instrumental|opening|interlude|hook|refrain)\b[:]?\}?',
    '', regex=True
)
df['lyrics'] = df['lyrics'].str.replace(r'\s+', ' ', regex=True).str.strip() # multiple spaces => single space

In [151]:
df['lyrics'][9]

'"If I only could I\'d be running up that hill If I only could I\'d be running up that hill" It doesn\'t hurt me You want to feel how it feels? You want to know know that it doesn\'t hurt me? You want to hear about the deal I\'m making? You be running up that hill You and me be running up that hill And if I only could Make a deal with God I\'d get him to swap our places Be running up that road Be running up that hill Be running up that building If I only could You don\'t want to hurt me But see how deep the bullet lies Unaware that I\'m tearing you asunder There\'s a thunder in our hearts baby So much hate for the ones we love Tell me we both matter don\'t we? You be running up that hill You and me be running up that hill You and me won\'t be unhappy And if I only could Make a deal with God I\'d get him to swap our places Be running up that road Be running up that hill Be running up that building If I only could C\'mon baby c\'mon c\'mon darling Let me steal this moment from you now C\

In [152]:
nlp = spacy.load("en_core_web_sm")

# Source of file: https://raw.githubusercontent.com/sugatagh/E-commerce-Text-Classification/main/JSON/english_acronyms.json
with open("../data/english_acronyms.json", encoding="utf-8") as f:
    acronyms_dict = json.load(f)
acronyms_dict = {k.lower(): v.lower() for k, v in acronyms_dict.items()}

# Source of file: https://www.kaggle.com/datasets/yetman/english-contractions
with open("../data/english_contractions.json", encoding="utf-8") as f:
    contractions_dict = json.load(f)
contractions_dict = {k.lower(): v.lower() for k, v in contractions_dict.items()}

In [153]:
def smart_tokenize(text):
    return re.findall(r"\w+(?:'\w+)?|[^\w\s]", text)

def convert_acronyms(text):
    tokens = smart_tokenize(text)
    result = []

    for token in tokens:
        key = token.lower()
        if key in acronyms_dict:
            expansion_words = acronyms_dict[key].split()
            expansion = ["I" if w == "i" else w for w in expansion_words]
            result.extend(expansion)
        else:
            result.append(token)
    return " ".join(result)


def convert_contractions(text):
    tokens = smart_tokenize(text)
    result = []

    for token in tokens:
        key = token.lower()
        if key in contractions_dict:
            expansion = contractions_dict[key].split()
            result.extend(expansion)
        else:
            result.append(token)
    return " ".join(result)


def clean_spacing(text):
    text = re.sub(r"\s+([.,!?;:])", r"\1", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [154]:
def preprocess_text(text):
    text = convert_acronyms(text)
    text = convert_contractions(text)
    doc = nlp(text)
    return clean_spacing(" ".join([t.text for t in doc]))

print(preprocess_text("IDK if you can't help Sofia!"))
print(preprocess_text("Idk if you'll be there."))
print(preprocess_text("idk if you'll be there."))
print(preprocess_text("Lol, we'll be there for sure!!"))

I do not know if you can not help Sofia!
I do not know if you will be there.
I do not know if you will be there.
laugh out loud, we will be there for sure!!


In [155]:
df['lyrics'] = df['lyrics'].apply(preprocess_text)
df

,artist,genre,title,album,lyrics,labels
0,Nirvana,Rock,You Know You’re Right,Nirvana,I will never bother you I will never promise t...,"[Calmness, Sadness]"
1,Damian Marley,Reggae,Here We Go,Stony Hill,Here we go My big ego is gon na get me in trou...,"[Power, Tension]"
2,The Mission UK,Rock,Jade,Another Fall from Grace,Adorned with Stars With her deft allusion and ...,"[Amazement, Calmness, Solemnity, Tenderness]"
3,UB40,Reggae,Food For Thought,Signing Off,Ivory Madonna dying in the dust Waiting for th...,"[Joyful activation, Sadness, Tension]"
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,I was toting my pack along the dusty Winnemucc...,"[Amazement, Calmness, Joyful activation]"
...,...,...,...,...,...,...
1105,Run-D.M.C.,Hip-Hop,King of Rock,King of Rock,i am the king of rock there is none higher Suc...,"[Joyful activation, Power]"
1106,The Beatles,Rock,Yesterday,Help!,Yesterday All my troubles seemed so far away N...,"[Nostalgia, Sadness, Tenderness]"
1107,The Sundays,Rock,Love,Blind,Picture myself as a thin white child Back to t...,"[Calmness, Power, Tenderness]"
1108,The Cure,Rock,Killing an Arab,Boys Don’t Cry,Standing on the beach With a gun in my hand St...,"[Power, Solemnity, Tension]"


In [156]:
df['lyrics'][9]

'" If I only could i would be running up that hill If I only could i would be running up that hill " It does not hurt me You want to feel how it feels? You want to know know that it does not hurt me? You want to hear about the deal i am making? You be running up that hill You and me be running up that hill And if I only could Make a deal with God i would get him to swap our places Be running up that road Be running up that hill Be running up that building If I only could You do not want to hurt me But see how deep the bullet lies Unaware that i am tearing you asunder there is a thunder in our hearts baby So much hate for the ones we love Tell me we both matter do not we? You be running up that hill You and me be running up that hill You and me will not be unhappy And if I only could Make a deal with God i would get him to swap our places Be running up that road Be running up that hill Be running up that building If I only could come on baby come on come on darling Let me steal this mom

In [159]:
df.to_csv("../data/MultiLabelSongs_EN_prepared.csv", index=False)